# Python exercise 'estimate gradient’ 

## Import Statements

In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random

from scipy import stats 
from sklearn.datasets import make_regression 

## Part 1

Pick three vectors from the table, using letters from your names and using each column.

In [16]:
# We pick C, K and A from Cathy and Kai
u = np.array([19., 41.]) # from C
v = np.array([86., 6.])  # from K
w = np.array([56., 20.]) # from A

Define the distance between two vectors as x, u as:
Dist (x,u)= (x1-u1)^2+(x2-u2)^2

Define the similarity function:
Diff(x) = ln(1+Dist(u,x))+ln(1+Dist(v,x))+ln(1+Dist(w,x))

Diff(x) is the distance between person x and your focus group of three people.



In [17]:
def dist(x,u):
    #return (x[0]-u[0])**2 + (x[1]-u[1])**2
    return np.sum((x - u) ** 2)

def diff(x):
    return np.log((1 + dist (u, x))) + np.log((1 + dist (v, x))) + np.log((1 + dist (w, x)))

## Part 2

Compute Diff(x) for 20 random points. 

What is the average value of Diff?
What is the lowest and highest value you found?


In [50]:
diffs = []

for i in range(20):
    #based on the values in the table I decided to pick a random number between 0 and 100
    diffs.append(diff(np.array([float(random.randrange(100)), float(random.randrange(100))]))) 

print(diffs)
print(f"Average: {np.mean(diffs)}, Lowest Value: {min(diffs)}, Highest Value: {max(diffs)}")

[23.432541312892724, 27.2697312190068, 25.495868923511708, 24.115705859328756, 25.66897812793738, 26.021998594367712, 21.923568377587287, 25.753389577989367, 21.161884615870406, 20.203451456749683, 21.127786226948917, 21.411062587951918, 23.70277849050528, 19.561680342195643, 26.012782463313393, 23.128624296482254, 23.496383712659796, 19.815688294744128, 26.31414141777867, 26.456800081244168]
Average: 23.6037422989533, Lowest Value: 19.561680342195643, Highest Value: 27.2697312190068


**Is random search a good way to minimize this function?**

No.

We have to hope that we randomly stumple across the vector with the minimum distance in our sample. 

We have no way of verifying if the lowest value we found is the minimum value.



## Part 3

Set x0=(50,50). Compute Diff(x)=Diff(50,50).

In [53]:
x0 = np.array([50., 50.])
diff_x0 = diff(x0)

print(x0)
print(diff_x0)

[50. 50.]
21.873705514964627


## Part 4

Create a function to compute and print the gradient of Diff by using a small delta=0.001.

In [75]:
def gradient(x, delta=.001):
    start = diff(x)
    result = []
    for i in range(len(x)):
        #change each number in x by delta
        x[i] += delta
        step = diff(x)
        result.append((step - start)/delta)
        x[i] -= delta
    return np.array(result)

## Part 5

Use this function to print the gradient at (0,0), (100,0), (0,100) and (100,100). 

In [78]:
points = np.array([[0., 0.],[100., 0.],[0., 100.],[100., 100.]])

for point in points:
    print(f"Gradient at {point}\tis {gradient(point)}")


Gradient at [0. 0.]	is [-0.07340544 -0.05306046]
Gradient at [100.   0.]	is [ 0.17747654 -0.07856262]
Gradient at [  0. 100.]	is [-0.03222729  0.05906304]
Gradient at [100. 100.]	is [0.02978588 0.05175347]


**Based on these values, where could be a minimum of Diff?**

The gradient at (0, 0), (0, 100) and (100, 100) is almost 0 so based on just the gradient any of these could be the minimum of Diff

Or more acculately, the gradient at nnonne of these points is 0 so none of them are the minimum.

## Part 6

Define xzero=(50,50) and stepsize=1.0. Make a function to compute xnext by taking e a small step into the opposite direction of the gradient, so that diff decreases. E.g. if the gradient is (0.4,0.6), do xnext=(oldx1-0.4*stepsize,oldx2-0.6*stepsize) 

In [69]:
def xnext(x,stepsize):
    start = gradient(x)
    xnext = np.array([x-(start[0]*stepsize),x-(start[1]*stepsize)])
    return xnext


xzero = np.array([50,50])
stepsize = 1
print(xnext(xzero,stepsize))

[[49.         49.        ]
 [48.97584206 48.97584206]]
